#### Imports

In [8]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [9]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory,'..','..','oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex

In [10]:
regex = r'SAW_06_02_2024_OIL_TRAFO_38_(\d+)\.S2P'

In [11]:
def read_s2p_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "TRAFO 38"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db[:,1,0]
            phase = s2p_file.s_deg[:,1,0]
            
            #Dataframe com informações selecionadas
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'OIL TYPE': 'MINERAL',
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S21 (db)' : s_parameters,
                'PHASE (DEG)': phase,
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            df = df.nlargest(3, 'S21 (db)')
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [12]:
df_mineral = read_s2p_data(regex, s2ps_path)

In [13]:
#assegura somente óleo de trafo38 lido
for name in df_mineral["OIL NAME"]:
    if name != "TRAFO 38":
        print("Diferente de TRAFO 38")

In [14]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_mineral.to_csv(path_save_csv + '/TRAFO_38_measurements.csv')